In [3]:
##### CREATING DECK
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

playing = True

# CARD CLASS

class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        return self.rank+" of "+self.suit

# DECK CLASS

class Deck:
    
    def __init__(self):
        
        self.deck = []  # start with an empty list
        
        for suit in suits:
            for rank in ranks:

                created_card = Card(suit,rank)
                self.deck.append(created_card)

    def __str__(self):
        deck_comp = ''
        for card in self.deck:
            deck_comp += '\n '+ card.__str__()
        return "The deck has:"+deck_comp
    
    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        single_card = self.deck.pop()
        return single_card

# HAND CLASS

class Hand:
    
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces
    
    def add_card(self, card):
            self.cards.append(card)
            self.value += values[card.rank]

            #track our aces
            if card.rank == 'Ace':
                self.aces += 1
    
    def adjust_for_ace(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

# CHIPS CLASS

class Chips:

    def __init__(self, total = 100):
        # This can be set to a default value or supplied by a user input
        self.total = total
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

# TAKE BET FUNCTION

def take_bet(chips):

    while True:

        try:
            chips.bet = int(input(f"You have {chips.total} chips available. How many chips would you like to bet?: "))
            
        except ValueError:
            print("Sorry, you must provide a integer")
            
        else:
            if chips.bet > chips.total:
                print (f"Sorry, your bet can't exceed {chips.total}.")
            elif chips.bet <= 0:
                print ("You can't bet zero or negative chips")
            else:
                break

# ADDITIONLA CHIPS FUNCTION

def check_and_add_chips(player_chips):
    if player_chips.total == 0:
        print("You have run out of chips!")
        
        while True:
            add_more = input("Would you like to add more chips to continue playing? Yes or No: ")
            if add_more[0].lower() == 'y':
                while True:
                    try:
                        additional_chips = int(input("How many chips would you like to add?"))
                        player_chips.total += additional_chips
                        print(f"You've added {additional_chips} chips. Your new total is {player_chips.total}.")
                        return True
                    except ValueError:
                        print("Please enter a valid number")
            elif add_more[0].lower() == 'n':
                print("Thank you for playing!")
                return False
            else:
                print("Sorry, you must enter 'Yes' or 'No'.")
    return True

# HIT FUNCTION

def hit(deck,hand):
    hand.add_card(deck.deal())
    hand.adjust_for_ace()

# HIT OR STAND FUNCTION

def hit_or_stand(deck,hand):
    global playing  # to control an upcoming while loop

    while True:
        x = input("Hit or stand? Enter h or s: ")
        
        if x[0].lower() == 'h':
            hit(deck,hand)
    
        elif x[0].lower() == 's':
            print("Player stands, dealer's turn")
            playing = False

        else:
            print("Sorry, something's wrong! Enter h or s")
            continue
        break

# DISPLAY CARDS FUNCTION

def show_some(player,dealer):

    #Show only one of a Dealer's cards
    print("\n Dealer's Hand: ")
    print("First card is hidden!")
    print(dealer.cards[1])

    #Show player's cards
    print("\n Player's hand: ")
    for card in player.cards:
        print(card)
        
def show_all(player,dealer):
    
        #Show all Dealer's cards
    
    print("\n Dealer's hand: ")
    for card in dealer.cards:
        print(card)
        
    print(f"Value of Dealer's hand is: {dealer.value}")
    
        #Show all the player cards
    
    print("\n Player's hand:")
    for card in player.cards:
        print(card)

    print(f"Value of Player's hand is: {player.value}")

# END OF GAME SCENERIOS

def player_busts(player,dealer,chips):
    print("BUST PLAYER")
    chips.lose_bet()

def player_wins(player,dealer,chips):
    print("PLAYER WINS!")
    chips.win_bet()

def dealer_busts(player,dealer,chips):
    print("PLAYER WINS! DEALER BUSTED!")
    chips.win_bet()
    
def dealer_wins(player,dealer,chips):
    print("DEALER WINS!")
    chips.lose_bet()
    
def push(player,dealer):
    print('Dealer and player tie! PUSH')

# Game logic

# Set up the Player's chips (first time starting chips)
player_chips = Chips()

while True:

    # Print an opening statement
    print("Welcome to BlackJack!!")
    
    # Create & shuffle the deck, deal two cards to each player
    deck = Deck()
    deck.shuffle()

    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # Check if player wants to continue or add more chips
    if not check_and_add_chips(player_chips):
        break  # End game if player doesn't want to add more chips and has no chips
    
    # Prompt the Player for their bet
    take_bet(player_chips)
    
    # Show cards (but keep one dealer card hidden)
    show_some(player_hand, dealer_hand)
    
    while playing:  # recall this variable from our hit_or_stand function
        # Prompt for Player to Hit or Stand
        hit_or_stand(deck, player_hand)
        
        # Show cards (but keep one dealer card hidden)
        show_some(player_hand, dealer_hand)
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        if player_hand.value > 21:
            player_busts(player_hand, dealer_hand, player_chips)
            break

    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
    if player_hand.value <= 21:
        while dealer_hand.value <= 17:
            hit(deck, dealer_hand)
    
        # Show all cards
        show_all(player_hand, dealer_hand)
        
        # Run different winning scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand, dealer_hand, player_chips)
        else:
            push(player_hand, dealer_hand)

    # Inform Player of their chips total 
    print(f'\nPlayer total chips are at: {player_chips.total}')
    
    # Call the function to check and add chips if needed
    if not check_and_add_chips(player_chips):
        break  # End the game if the player doesn't want to add more chips
    
    # Ask to play again only if player hasn't added chips
    playing = True
    if player_chips.total > 0:
        new_game = input("Would you like to play again? Yes or No: ")
        if new_game[0].lower() == 'y':
            print("Ok, let's go!")
        else:
            print("Thank you for playing!")
            break

Welcome to BlackJack!!


You have 100 chips available. How many chips would you like to bet?:  100



 Dealer's Hand: 
First card is hidden!
Ten of Clubs

 Player's hand: 
Six of Spades
King of Spades


Hit or stand? Enter h or s:  h



 Dealer's Hand: 
First card is hidden!
Ten of Clubs

 Player's hand: 
Six of Spades
King of Spades
King of Clubs
BUST PLAYER

Player total chips are at: 0
You have run out of chips!


Would you like to add more chips to continue playing? Yes or No:  n


Thank you for playing!
